In [16]:
import numpy as np
import random

ROWS = 5
COLUMNS = 5

WIN_STATES = []     #Creating a list for win_states
for x in range(5):
  for y in range(5):
    WIN_STATES.append((x, y))

WIN_STATES.remove((0,0))
WIN_STATES.remove((1,2))
WIN_STATES.remove((1,3))
WIN_STATES.remove((1,4))
WIN_STATES.remove((1,1))
WIN_STATES.remove((2,1))
WIN_STATES.remove((2,2))
WIN_STATES.remove((2,3))
WIN_STATES.remove((3,1))
WIN_STATES.remove((3,2))
WIN_STATES.remove((3,3))
WIN_STATES.remove((4,2))
WIN_STATES.remove((4,4))

print("WIN_STATES:",WIN_STATES)


LOSE_STATES = []      #Creating list for losing_states
for x in range(5):
  for y in range(5):
    LOSE_STATES.append((x, y))
    
LOSE_STATES.remove((0,1))
LOSE_STATES.remove((0,2))
LOSE_STATES.remove((0,3))
LOSE_STATES.remove((0,4))
LOSE_STATES.remove((1,0))
LOSE_STATES.remove((1,1))
LOSE_STATES.remove((1,2))
LOSE_STATES.remove((1,3))
LOSE_STATES.remove((2,0))
LOSE_STATES.remove((2,1))
LOSE_STATES.remove((2,2))
LOSE_STATES.remove((2,3))
LOSE_STATES.remove((2,4))
LOSE_STATES.remove((3,0))
LOSE_STATES.remove((3,1))
LOSE_STATES.remove((3,2))
LOSE_STATES.remove((3,3))
LOSE_STATES.remove((3,4))
LOSE_STATES.remove((4,0))
LOSE_STATES.remove((4,1))
LOSE_STATES.remove((4,3))

print("LOSE_STATES:",LOSE_STATES)
START = (1,1)     #Defining start state at (1,1)
DETERMINISTIC = True

class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLUMNS])
        self.board[4,4] = -1
        self.board[4,2] = -1
        self.board[1,4] = -1
        self.board[0,0] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC

    def giveReward(self):
        if self.state in WIN_STATES:
            return 1
        elif self.state in LOSE_STATES:
            return -1
        else:
            return 0
    def isEndFunc(self):
        if (self.state in WIN_STATES) or (self.state in LOSE_STATES):
            self.isEnd = True
    def nxtPosition(self, action):
        if self.determine:
            if action == "N":
                nxtState = (self.state[0] , self.state[1]- 1)
            elif action == "S":
                nxtState = (self.state[0], self.state[1] + 1)
            elif action == "W":
                nxtState = (self.state[0] - 1, self.state[1])
            else:
                nxtState = (self.state[0] + 1 , self.state[1])
            
            if (nxtState[0] >= 0) and (nxtState[0] <= 4):
                if (nxtState[1] >= 1) and (nxtState[1] <= 3):
                    if nxtState != (0,0):
                        return nxtState
                    if nxtState != (4,4):
                        return nxtState
                    if nxtState != (4,2):
                        return nxtState
                    if nxtState != (1,4):
                        return nxtState
            return self.state
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLUMNS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')

class Agent:          #Creating an agent for the player

    def __init__(self):
        self.states = []
        self.actions = ["N", "S", "W", "E"]
        self.State = State()
        self.lr = 0.5
        self.exp_rate = 0.5

        self.state_values = {}      #Defining rewards
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLUMNS):
                #self.state_values[(i, j)] = 0  # Setting initial value to 0 #Q2
                self.state_values[(i,j)] = random.random() # Setting initial value randomly [0,1) #Q1

    def chooseAction(self):       # Defining the agent to choose action with the most expected value
        mx_nxt_reward = 0
        action = ""

        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:       #Greedy algo stage            
            for a in self.actions:        #deterministic Action
                nxt_reward = self.state_values[self.State.nxtPosition(a)]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
        return action

    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        return State(state=position)

    def reset(self):
        self.states = []
        self.State = State()

    def play(self, rounds=5):
        i = 0
        while i < rounds:
            if self.State.isEnd:    #Backpropagation stage                
                reward = self.State.giveReward()
                self.state_values[self.State.state] = reward  
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    reward = self.state_values[s] + self.lr * (reward - self.state_values[s])
                    self.state_values[s] = round(reward, 1)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()      #traces appending
                self.states.append(self.State.nxtPosition(action))
                print("current position {} action {}".format(self.State.state, action))
                self.State = self.takeAction(action)
                self.State.isEndFunc()
                print("Next state", self.State.state)
                print("---------------------")

    def showValues(self):
        for i in range(0, BOARD_ROWS):
            print('----------------------------------')
            out = '| '
            for j in range(0, BOARD_COLUMNS):
                out += str(self.state_values[(i, j)]).ljust(6) + ' | '
            print(out)
        print('----------------------------------')


if __name__ == "__main__":
    ag = Agent()
    ag.play(50)
print(ag.showValues())

WIN_STATES: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (2, 0), (2, 4), (3, 0), (3, 4), (4, 0), (4, 1), (4, 3)]
LOSE_STATES: [(0, 0), (1, 4), (4, 2), (4, 4)]
current position (1, 1) action N
Next state (1, 1)
---------------------
current position (1, 1) action N
Next state (1, 1)
---------------------
current position (1, 1) action N
Next state (1, 1)
---------------------
current position (1, 1) action N
Next state (1, 1)
---------------------
current position (1, 1) action W
Next state (0, 1)
---------------------
Game End Reward 1
current position (1, 1) action W
Next state (0, 1)
---------------------
Game End Reward 1
current position (1, 1) action W
Next state (0, 1)
---------------------
Game End Reward 1
current position (1, 1) action S
Next state (1, 2)
---------------------
current position (1, 2) action E
Next state (2, 2)
---------------------
current position (2, 2) action E
Next state (3, 2)
---------------------
current position (3, 2) action W
Next state (2, 2)
----------